In [24]:
import numpy as np
import pandas as pd
import random
import pickle

import flair
from flair.data import Sentence
from flair.embeddings import TransformerDocumentEmbeddings

from sklearn.metrics.pairwise import cosine_similarity

In [6]:
device = 'cpu' 
flair.device = device

In [7]:
%%markdown
## Подгрузка пиклов

## Подгрузка пиклов


In [8]:
with open('../pickles/popularities.pickle', 'rb') as f:
    popularities = pickle.load(f)

In [9]:
with open('../pickles/embeddings.pickle', 'rb') as f:
    embeddings = pickle.load(f)

In [10]:
with open('../pickles/documents.pickle', 'rb') as f:
    documents = pickle.load(f)

In [11]:
%%markdown
## Построение эмбедингов из истории запросов

## Построение эмбедингов из истории запросов


In [13]:
search_hist_df = pd.read_csv('../data/fixed_search_history.csv', engine='python', nrows=1000)
search_hist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   wbuser_id   1000 non-null   object
 2   UQ          1000 non-null   object
 3   cnt         1000 non-null   int64 
 4   locale      1000 non-null   object
 5   weekday     1000 non-null   int64 
 6   time        1000 non-null   object
dtypes: int64(3), object(4)
memory usage: 54.8+ KB


In [14]:
embedder = TransformerDocumentEmbeddings('sentence-transformers/stsb-xlm-r-multilingual', fine_tune=False)

In [16]:
%%time
test_sents = [Sentence(text) for text in search_hist_df['UQ'].values]
embedder.embed(test_sents)

CPU times: user 2min 3s, sys: 20.2 s, total: 2min 23s
Wall time: 26.1 s


[Sentence: "тапочки женские домашние"   [− Tokens: 3],
 Sentence: "чехол для бейджика"   [− Tokens: 3],
 Sentence: "guess"   [− Tokens: 1],
 Sentence: "штора для ванной"   [− Tokens: 3],
 Sentence: "фототфон"   [− Tokens: 1],
 Sentence: "туфли мужские осенние"   [− Tokens: 3],
 Sentence: "леггинсы женские"   [− Tokens: 2],
 Sentence: "чехол на айфон 11"   [− Tokens: 4],
 Sentence: "ботинки женские мембрана"   [− Tokens: 3],
 Sentence: "обувь женская оксфорды"   [− Tokens: 3],
 Sentence: "противоударный чехол на айфон 6"   [− Tokens: 5],
 Sentence: "зимняя обувь для мальчиков 37-38 размеров"   [− Tokens: 6],
 Sentence: "бисер"   [− Tokens: 1],
 Sentence: "робот змея"   [− Tokens: 2],
 Sentence: "костюм на флисе женский"   [− Tokens: 4],
 Sentence: "светящийся ошейник для собак"   [− Tokens: 4],
 Sentence: "изделия из мед сплава"   [− Tokens: 4],
 Sentence: "ремкомплект раздатки нива chevrolet"   [− Tokens: 4],
 Sentence: "оружие на лёго человека"   [− Tokens: 4],
 Sentence: "костюмы жен

In [4]:
%%markdown
## Построение топа запросов

## Построение топа запросов


In [34]:
TOP_COUNT = 20

In [102]:
test = random.choice(test_sents)
test.to_plain_string()

'зимние штаны балоневые'

In [103]:
distances = cosine_similarity(test.embedding[np.newaxis, :], embeddings)[0]
distances = distances * popularities
distances.shape

(320694,)

In [104]:
ratings = np.argsort(-distances).tolist()[:TOP_COUNT]

In [105]:
print(f'Рейтинг подходящих запросов для: "{test.to_plain_string()}"')
pd.DataFrame({'request': list(map(documents.__getitem__, ratings)),
              'score': distances[ratings], 'popularity': popularities[ratings]})

Рейтинг подходящих запросов для: "зимние штаны балоневые"


,request,score,popularity
0,штаны зимние балоневые утепленные,0.967899,1.000000
1,зимние балоневые штаны женские,0.965382,1.000000
2,зимние штаны для девочки балоневые,0.959173,1.000000
3,балоневые штаны для девочки зимние,0.952406,1.000000
4,штаны зимние женские,0.908197,1.000000
5,штаны зимние для девочки балоневые,0.906603,0.944444
6,штаны для девочки зимние балоневые,0.903178,0.944444
7,брюки женские зимние на синтепоне,0.903085,1.000000
8,брюки зимние,0.900746,1.000000
9,штаны женские зимние,0.898483,1.000000
